<a href="https://colab.research.google.com/github/qq1499412503/data_A2/blob/master/Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function
import keras
from keras.optimizers import SGD
from keras.layers import add, Flatten, MaxPooling2D, Dropout, ZeroPadding2D
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
import numpy as np
import os

Using TensorFlow backend.


# data pre-processing

Loading data from keras lib

In [0]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print info to show data info

In [0]:
def print_info(x_train, y_train, x_test):
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')
    print('y_train shape:', y_train.shape)

preprocessing data based on normalization and subtract pixel mean

In [0]:
def normalizer(x_train, x_test):
    x_train = x_train.astype('float32') / 255
    x_test = x_test.astype('float32') / 255
    return x_train, x_test

In [0]:
def subtract_pixel_mean(x_train, x_test):
    x_train_mean = np.mean(x_train, axis=0)
    x_train -= x_train_mean
    x_test -= x_train_mean
    return x_train, x_test

normallize the laber by converting class vectors to binary class matrices

In [0]:
def label_binary(y_train, y_test, num_classes):
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)
    return y_train, y_test

print and see the changes before and after pre-processing

In [0]:
print_info(x_train, y_train, x_test)
(x_train, x_test)=normalizer(x_train, x_test)
#(x_train, x_test)=subtract_pixel_mean(x_train, x_test)
(y_train, y_test)=label_binary(y_train, y_test, 10)
print_info(x_train, y_train, x_test)

# Modeling_Resnet

create basic BN_layers

In [0]:
#resnet default input use 224x224 image size, but cifar-10 only have 32x32
#based on refercened atrical in report, The numbers of 
#filters are {16, 32, 64} respectively
#here init by 16 with s 3×3 convolutions
num_filters=16
kernel_size=3
strides=1

def resnet_layer(inputs,
                 num_filters=num_filters,
                 kernel_size=kernel_size,
                 strides=strides,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):

    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x


structure model

In [0]:
#model created by resnet following the structure in report
#num_filters start with 16 and 
#num of blocks based on (depth-2)/6 which can be [20, 32, 44...]
def resnet_model(input_shape, depth=32, num_classes=10):
    num_filters = 16
    if (depth - 2) % 6 != 0:
        raise ValueError('depth should be 6n+2 (eg 20, 32, 44 in [a])')
    num_res_blocks = int((depth - 2) / 6)
    inputs = Input(shape=input_shape)
    x = resnet_layer(inputs=inputs)
    for stack in range(3):
        for res_block in range(num_res_blocks):
            strides = 1
            if stack > 0 and res_block == 0: 
                strides = 2  
            y = resnet_layer(inputs=x,
                             num_filters=num_filters,
                             strides=strides)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters,
                             activation=None)
            if stack > 0 and res_block == 0:  
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])
            x = Activation('relu')(x)
        num_filters *= 2

    # Add classifier on top.
    # v1 does not use BN after last shortcut connection-ReLU
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model
    

In [0]:
#a example of resnet 32 is showing below
def resnet_model_32(input_shape, depth=32, num_classes=10):
    num_filters = 16
    num_res_blocks = int((depth - 2) / 6)
    inputs = Input(shape=input_shape)
    x = resnet_layer(inputs=inputs)

#starting build res_32 as example in 32 layers    
#stack 0
    #L1
    y = resnet_layer(inputs=x,
                     num_filters=16,
                     strides=1)
    y = resnet_layer(inputs=y,
                     num_filters=16,
                     activation=None)
    #L2
    y = resnet_layer(inputs=x,
                     num_filters=16,
                     strides=1)
    y = resnet_layer(inputs=y,
                     num_filters=16,
                     activation=None)
    #L3
    y = resnet_layer(inputs=x,
                     num_filters=16,
                     strides=1)
    y = resnet_layer(inputs=y,
                     num_filters=16,
                     activation=None)
    #L4
    y = resnet_layer(inputs=x,
                     num_filters=16,
                     strides=1)
    y = resnet_layer(inputs=y,
                     num_filters=16,
                     activation=None)
    #L5
    y = resnet_layer(inputs=x,
                     num_filters=16,
                     strides=1)
    y = resnet_layer(inputs=y,
                     num_filters=16,
                     activation=None)
    
#stack 1
    #L1
    y = resnet_layer(inputs=x,
                     num_filters=32,
                     strides=2)
    y = resnet_layer(inputs=y,
                     num_filters=32,
                     activation=None)
    x = resnet_layer(inputs=x,
                 num_filters=num_filters,
                 kernel_size=1,
                 strides=strides,
                 activation=None,
                 batch_normalization=False)
    #L2
    y = resnet_layer(inputs=x,
                     num_filters=32,
                     strides=1)
    y = resnet_layer(inputs=y,
                     num_filters=32,
                     activation=None)
    #L3
    y = resnet_layer(inputs=x,
                     num_filters=32,
                     strides=1)
    y = resnet_layer(inputs=y,
                     num_filters=32,
                     activation=None)
    #L4
    y = resnet_layer(inputs=x,
                     num_filters=32,
                     strides=1)
    y = resnet_layer(inputs=y,
                     num_filters=32,
                     activation=None)
    #L5
    y = resnet_layer(inputs=x,
                     num_filters=32,
                     strides=1)
    y = resnet_layer(inputs=y,
                     num_filters=32,
                     activation=None)
#stack 2
    #L1
    y = resnet_layer(inputs=x,
                     num_filters=64,
                     strides=2)
    y = resnet_layer(inputs=y,
                     num_filters=64,
                     activation=None)
    x = resnet_layer(inputs=x,
                 num_filters=num_filters,
                 kernel_size=1,
                 strides=strides,
                 activation=None,
                 batch_normalization=False)
    #L2
    y = resnet_layer(inputs=x,
                     num_filters=64,
                     strides=1)
    y = resnet_layer(inputs=y,
                     num_filters=64,
                     activation=None)
    #L3
    y = resnet_layer(inputs=x,
                     num_filters=64,
                     strides=1)
    y = resnet_layer(inputs=y,
                     num_filters=64,
                     activation=None)
    #L4
    y = resnet_layer(inputs=x,
                     num_filters=64,
                     strides=1)
    y = resnet_layer(inputs=y,
                     num_filters=64,
                     activation=None)
    #L5
    y = resnet_layer(inputs=x,
                     num_filters=64,
                     strides=1)
    y = resnet_layer(inputs=y,
                     num_filters=64,
                     activation=None)
         
            x = keras.layers.add([x, y])
            x = Activation('relu')(x)
        num_filters *= 2

    # Add classifier on top.
    # v1 does not use BN after last shortcut connection-ReLU
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model

Training the model

In [0]:
#model would be training based on irrationable learning rate on both SGD 
#and adam optmizer
def lr_schedule(epoch):
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

input_shape = x_train.shape[1:]
model = resnet_model(input_shape=input_shape, depth=32)
model.compile(loss='categorical_crossentropy',
              optimizer=SGD(lr=lr_schedule(0)),
              metrics=['accuracy'])

In [0]:
#training in specific batch_size, 32 is better for both training speed 
#and loss decreasing
models=model.fit(x_train, y_train,
              batch_size=32,
              epochs=100,
          validation_data=(x_test, y_test),
              )

# Evaluating

Classification report

In [0]:
#classification_report will be produced to show the f-score of each class 
#and its recall value
predictions = model.predict(x_test, batch_size=16)
print('[INFO] Evaluating model')
from sklearn.metrics import classification_report
result = classification_report(y_test.argmax(axis=1), predictions.argmax(axis=1),
                            target_names=['airplane', 'automobile', 'bird',
                                          'cat','deer','dog','frog','horse','ship','truck'])
print(result)

Confusion matrix

In [0]:
#confusion matrix will be used to present the ability to class the image for 
#each class
#label follow by the order of target_names=['airplane', 'automobile', 'bird',
#                            'cat','deer','dog','frog','horse','ship','truck']
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test.argmax(axis=1), predictions.argmax(axis=1))
print(cm)

Training loss

In [0]:
# loss in both training and test sets will be plot following
#loss is able to show the changes of model in performance


import matplotlib.pyplot as plt

N = np.arange(0, 200)
plt.style.use('ggplot')
plt.figure()
plt.plot(N, models.history['loss'], label='train_loss')
plt.plot(N, models.history['val_loss'], label='val_loss')
plt.plot(N, models.history['acc'], label='train_acc')
plt.plot(N, models.history['val_acc'], label='val_acc')
plt.title('Training loss and Accuracy')
plt.xlabel('epoch #')
plt.ylabel('loss/accuracy')
plt.ylim((0, 10))
plt.legend()
#name2='img'
#plt.savefig(str(name2)+'.png')

In [0]:
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'cifar10_%s_model.{epoch:03d}.h5' % model_type
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)
models.save(filepath)